**NOTE:**  

I recently developed a tutorial based upon this notebook.  You can find that tutorial [here](https://publiclab.org/notes/bsugar/03-30-2019/cultivating-community-through-cultivating-data/)  
However, if you are interested in the code I used to make the entire diagram, you can find that in the last cell below.
    

In [ ]:
from IPython.display import Image

SCHEMA: https://github.com/publiclab/plots2/blob/master/db/schema.rb.example
RAW TEXT: https://raw.githubusercontent.com/publiclab/plots2/master/db/schema.rb.example
RAILS ASSOCIATIONS: https://guides.rubyonrails.org/v2.3.11/association_basics.html
MODELS: https://github.com/publiclab/plots2/tree/master/app/models
OVERALL: https://www.holistics.io/blog/a-database-diagram-designer-built-for-developers-and-analysts/
RAILS: https://www.holistics.io/blog/visualize-rails-database-to-erd-diagram/
MANY TO MANY: https://www.holistics.io/blog/dbdiagram-io-many-to-many-relationship-diagram-generator-script/

In [ ]:
Image(filename='test.png')

In [ ]:
GOT RID OF RUSERS

In [1]:
USE TABLE AS
has_many :revision, foreign_key: 'nid', dependent: :destroy

JUST KNOW ITS DIFFERENT
has_many :drupal_upload, foreign_key: 'nid' # , dependent: :destroy # re-enable in Rails 5

IS TERM_NODE
  has_many :node_tag, foreign_key: 'nid' # , dependent: :destroy # re-enable in Rails 5
  has_many :comments, foreign_key: 'nid', dependent: :destroy # re-enable in Rails 5
  has_many :drupal_content_type_map, foreign_key: 'nid' # , dependent: :destroy # re-enable in Rails 5
  has_many :drupal_content_field_mappers, foreign_key: 'nid' # , dependent: :destroy # re-enable in Rails 5
  has_many :drupal_content_field_map_editor, foreign_key: 'nid' # , dependent: :destroy # re-enable in Rails 5
  has_many :images, foreign_key: :nid
  has_many :node_selections, foreign_key: :nid, dependent: :destroy
  has_many :answers, foreign_key: :nid, dependent: :destroy

https://guides.rubyonrails.org/v2.3.11/association_basics.html ("2.4 The has_many :through Association")

BREAK DOWN INTO THREE
has_many :drupal_files, through: :drupal_upload
SINCE WE ARE GOING *THROUGH* DRUPAL UPLOAD, WE START THERE
IN THIS CASE WE ALREADY HAD IT ABOVE, NOW WE GO FROM UPLOAD TO FILES
Ref: node.id > upload.nid
THEN WE GO FROM UPLOAD TO THE FILE
            
has_many :tag, through: :node_tag

belongs_to :user, foreign_key: 'uid'

SyntaxError: invalid syntax (<ipython-input-1-8a67d4f49c18>, line 1)

In [ ]:
https://guides.rubyonrails.org/v2.3.11/association_basics.html ("2.4 The has_many :through Association")

BREAK DOWN INTO THREE
has_many :drupal_files, through: :drupal_upload
            
SINCE WE ARE GOING *THROUGH* DRUPAL UPLOAD, WE START THERE
IN THIS CASE WE ALREADY HAD IT ABOVE, BUT LET'S BRING IT DOWN BELOW TO KEEP TRACK.

STEP 1: FROM UPLOAD TO FILES
Ref: node.id > upload.nid
    
STEP 2: I'M GOING TO KEEP UP HERE
Ref: upload.nid > files.id


Table "node" {
  "id" varchar [primary key]
  "vid" integer
  "type" string
  "language" string
  "title" string
  "uid" integer
   ...
  "precision" integer
}

// HAS_MANY THROUGH
Ref: node.id > upload.nid
Ref: upload.nid > files.id


Table "upload" {
  "id" varchar
  "fid" integer
  "nid" integer
   ...
  "weight" integer
}



Table "files" {
  "id" varchar
  "uid" integer
   ...
  "timestamp" integer
}

In [ ]:
SINCE I WANTED TO SHOW "THROUGH" WITH A LITTLE MORE CLARITY BUT HAVING A LINE THAT ACTUALLY
WENT THROUGH, I ACTUALLY DID A THREE BASED CONNECTION OF:

MANY TO MANY
ONE TO ONE
MANY TO ONE

FOR EXAMPLE, I CONCEIVED OF THIS LINE FROM THE USERS MODEL AS:
https://github.com/publiclab/plots2/blob/856c371a5bbba75d9838ca27f4fa6cf0f0c59ece/app/models/user.rb#L57



LIKE THIS:
    
// USERS
// has_many :revisions, through: :node
    
// NORMALLY YOU'D THINK OF THIS AS A USER HAVING A VID BUT IT DOESN'T SO
// WE NEED TO "GET IT ONE" IN A SENSE LIKE THIS

// Ref: "users.vid" < "node"."vid" < "node_revisions"."id"

// INSTEAD WE'LL CONNECT USER TO NODE AND NODE TO NODE REVISIONS

Ref: users.id > node.uid  //FOUND ABOVE IN NODE SECTION
Ref: node.uid - node.vid
Ref: node.vid < node_revisions.id

// THIS AFFORDED ME A TINY LOOP TO SHOW THE "THROUGH"

In [ ]:
NOW LET'S DO THE SAME FOR: has_many :tag, through: :node_tag
STEP 1: WE'LL GO FROM THE NODE TO NODE TAG FIRST
Ref: node.id > node_tag.nid

STEP TWO:
Ref: node_tag.nid > tag.id
    
Table "node" {
  "id" varchar [primary key]
  "vid" integer
  "type" string
  "language" string
  "title" string
  "uid" integer
   ...
  "precision" integer
}   
    
Table "term_data" as "tag" {
  "id" varchar
  "vid" integer
  "name" string
  "description" text
  "weight" integer
  "count" integer
  "parent" string
}

Table "term_node" as "node_tag" {
  "id" varchar
  "nid" integer
  "vid" integer
  "tid" integer
}
    
Table "community_tags" {
  "id" varchar
  "tid" integer
  "nid" integer
  "uid" integer
  "date" integer
}

Table "term_data"{
  "id" varchar
  "vid" integer
  "name" string
  "description" text
  "weight" integer
  "count" integer
  "parent" string
}

In [ ]:
BELONGS TO:  https://guides.rubyonrails.org/v2.3.11/association_basics.html 
("2.1 The belongs_to Association")

# SO A BELONGS TO IS A ONE TO ONE RECORD AS FAR AS THE EER DIAGRAM GOES
    
belongs_to :user, foreign_key: 'uid'
Ref: node.uid - users.id
    
Table "node" {
  "id" varchar [primary key]
  "vid" integer
  "type" string
  "language" string
  "title" string
  "uid" integer
   ...
  "precision" integer
}  

Ref: node.uid - users.id

Table "rusers" as "users"{
  "id" varchar
  "username" string
   ...
  "is_verified" boolean
}

In [2]:
UNFORTUNATLELY DBDIAGRAM.IO DOESN'T THROWS AN ERROR IF YOU HAVE A DOUBLE CONNECTION.
FOR EXAMPLE, IN THE CASE WHERE THE MODEL CONNECTS FROM A MANY TO ONE, THE MODEL THAT IS 
THE "ONE" OF THE "MANY TO ONE" MIGHT HAVE A "ONE TO MANY" BACK TO THE SAME MODEL. DBDIAGRAM
WILL THROW AN ERROR ABOUT THIS.  SO YOU NEED TO KEEP TRACK OF WHAT YOU'VE ALREADY DONE.

SINCE I WAS ONLY INTERESTED IN USERS, TAGS, COMMUNICATION (QUESTIONS, ANSWERS, COMMENTS, 
ETC.) I ONLY PAIRED DOWN THE EER DIAGRAM TO THOSE MODELS.  NOTE: I ONLY RECENTLY LEANRED
ABOUT THE TABLE ALIASES SO I MADE SOME CHANGES TO THE ACTUAL TABLES.  IF THERE WAS AN
INSTANCE WHERE THERE WAS A REPETITION AS DESCRIBED ABOVE, I STILL TRIED TO MARK IT.

ARRANGE THE DIAGRAM HOW YOU'D LIKE SINCE I WANTED THE FOUR I MENTIONED, I TRIED TO GROUP THEM
ACCORDING TO THOSE ASSOCIATIONS.

THEN EXPORT --> PDF WHEN YOU OPEN IT, YOU'LL SEE TWO FILES, ONE IS JUST THE GRAY LINES, THE
OTHER, BLUE WITH THE INDICATORS OF THE DIRECTIONALITY (E.G., MANY TO MANY)  I USED PREVIEW
TO DELETE THE BLUE ONE.  I DIDN'T REALIZE IT'S SIGNIFICANCE AT THE TIME SO I JUST USED THE
GREY ONE, WHICH I FEEL ACTUALLY HAS AN ADVANTAGE WHEN 

SyntaxError: invalid syntax (<ipython-input-2-c0b611d4d0eb>, line 1)

In [ ]:
https://github.com/publiclab/plots2/blob/master/app/models/node.rb

In [3]:
IF YOUR FILE IS HORIZONTAL: 
FILE >> IMPORT
CHOOSE PDF (PLOTS_DB_EER_GRAY_LINES_ONLY.PDF)
I CHOOSE THE LOWEST FINE AT 130
UNGROUP, DELETE HOLISTICS IN THE CORNER
FILE >> DOCUMENT PROPERTIES, CHANGE TO LETTER
RIGHT BELOW THE DOCUMENT SIZES (E.G., A4) IS PROTRAIT AND LANDSCAPE RADIO BUTTONS.
ALSO, CHOOSE LETTER (OR SOME LARGER SIZE, I'D PROBABLY GO LARGER)
OPEN UP THE ARROW THAT SAYS RESIZE PAGE TO CONTENT >> RESIZE PAGE TO DRAWING OR SELECTION
NOW EITHER SAVE AS AND SAVE AS SVG OR
CLICK ON TOP BOXES AND PATHS AND COLOR!
FYI: BELOW, WHERE THE COLORS ARE, THERE'S A SCROLL BAR UNDERNEATH

SyntaxError: invalid syntax (<ipython-input-3-009e02e896de>, line 1)

In [ ]:
I FOUND FIGMA TO NOT WORK SO GREAT.  IF YOU IMPORTED THE LETTER VERSION THEN IT WOULDN"T SHOW
IF YOU IMPORTED THE SMALLER VERSION, WHEN YOU ENLARGED IT, THE TEXT DID NOT ENLAGE.  EVEN SO
BEFORE I DISCOVERED THIS I HAD WRITTEN EVERYTHING DOWN SO HERE YOU GO!

FILE >> SAVE AS >> PLAIN SVG 
OPEN FIGMA
FOR SOME REASON YOU NEED TO CREATE THE FILE FIRST, CAN'T JUST IMPORT AND SVG
OPEN NEW FILE
RENAME
DRAG SVG FILE ONTO BLANK BACKGROUND
CHANGE BACKGROUND COLOR TO WHITE
I MADE THINGS HAVING TO DO WITH EACH TYPE OF DATA A DIFFERNT COLOR
CLICK ON BLUE RECTANGLE, CHANGE FILL ON THE RIGHT
MADE USER_TAGS AND LIKES AN INBETWEEN COLOR
THEN THE LINES, MADE THE COLOR EQUAL THE STARTING POINT OR ANY EQUIVALENT E.G. NID'S ALL
ORANGE
THICKNESS 2
ALSO LINED UP PATHS SO WHEN THEY WERE OF THE SAME COLOR THEY DIDN"T OVER LAP
NOT SURE WHY BUT DASHES IS NOT A CHOICE UNDER ADVANCED STROKE
SELECT EVERYTHIHG, CMD+G (RIGHT CLICK GROUP)
RIGHT CLICK, FRAME SELECTION

In [ ]:
Table "answer_selections" {
  "id" varchar
  "user_id" integer
  "aid" integer
  "liking" boolean
  "following" boolean
}

Table "answers" {
  "id" varchar
  "uid" integer
  "nid" integer
  "content" text
  "cached_likes" integer
  "created_at" datetime
  "updated_at" datetime
  "accepted" boolean
}

Table "comments" {
  "id" varchar
  "pid" integer
  "nid" integer
  "uid" integer
  "subject" string
  "comment" text
  "hostname" string
  "timestamp" integer
  "status" integer
  "format" integer
  "thread" string
  "name" string
  "mail" string
  "homepage" string
  "aid" integer
  "comment_via" integer
  "message_id" string
}

Table "community_tags" {
  "id" varchar
  "tid" integer
  "nid" integer
  "uid" integer
  "date" integer
}


// Table "files" {
//   "id" varchar
//   "uid" integer
//   "filename" string
//   "filepath" string
//   "filemime" string
//   "filesize" integer
//   "status" integer
//   "timestamp" integer
// }


Table "images" {
  "id" varchar
  "title" string
  "uid" integer
  "nid" integer
  "notes" string
  "version" integer
  "photo_file_name" string
  "photo_content_type" string
  "photo_file_size" string
  "created_at" datetime
  "updated_at" datetime
  "remote_url" string
  "vid" integer
}



Table "likes" {
  "id" varchar
  "likeable_id" integer
  "user_id" integer
  "likeable_type" string
  "created_at" datetime
  "updated_at" datetime
  "emoji_type" string
}

Table "node" {
  "id" varchar
  "vid" integer
  "type" string
  "language" string
  "title" string
  "uid" integer
  "status" integer
  "created" integer
  "changed" integer
  "comment" integer
  "promote" integer
  "moderate" integer
  "sticky" integer
  "tnid" integer
  "translate" integer
  "cached_likes" integer
  "comments_count" integer
  "drupal_node_revisions_count" integer
  "path" string
  "main_image_id" integer
  "slug" string
  "legacy_views" integer
  "views" integer
  "latitude" decimal
  "longitude" decimal
  "precision" integer
}



Table "node_revisions" {
  "id" varchar
  "nid" integer
  "uid" integer
  "title" string
  "body" text
  "teaser" text
  "log" text
  "timestamp" integer
  "format" integer
  "status" integer
}

Table "node_selections" {
  "id" varchar
  "user_id" integer
  "nid" integer
  "following" boolean
  "liking" boolean
}

Table "relationships" {
  "id" varchar
  "follower_id" integer
  "followed_id" integer
  "created_at" datetime
  "updated_at" datetime
}


Table "tag_selections" {
  "id" varchar
  "user_id" integer
  "tid" integer
  "following" boolean
}

Table "term_data" {
  "id" varchar
  "vid" integer
  "name" string
  "description" text
  "weight" integer
  "count" integer
  "parent" string
}

Table "term_node" {
  "id" varchar
  "nid" integer
  "vid" integer
  "tid" integer
}

// Table "upload" {
//   "id" varchar
//   "fid" integer
//   "nid" integer
//   "vid" integer
//   "description" string
//   "list" integer
//   "weight" integer
// }

Table "user_selections" {
  "id" varchar
  "self_id" integer
  "other_id" integer
  "following" boolean
}

Table "user_tags" {
  "id" varchar
  "uid" integer
  "value" string
  "created_at" datetime
  "updated_at" datetime
  "data" text
}

Table "users" {
  "id" varchar
  "name" string
  "pass" string
  "mail" string
  "mode" integer
  "sort" integer
  "threshold" integer
  "theme" string
  "signature" string
  "signature_format" integer
  "created" integer
  "access" integer
  "login" integer
  "status" integer
  "timezone" string
  "language" string
  "picture" string
  "init" string
  "data" text
  "timezone_id" integer
  "timezone_name" string
  "lat" decimal
  "lon" decimal
  // "active" integer
  // "passive" integer
}

// NODE
Ref: "node"."id" < "node_revisions"."nid"
Ref: "node"."id" < "community_tags"."nid"
Ref: "node"."id" < "comments"."nid"
Ref: "node"."id" < "images"."nid"
Ref: "node"."id" < "node_selections"."nid"
Ref: "node"."id" < "answers"."nid"

//// THROUGH
//// has_many :drupal_files, through: :drupal_upload
//// Ref: "node"."fid" < "upload".fid < "files"."id"
// Ref: "node"."id" < "upload"."nid"
// Ref: "upload"."nid" - "upload"."fid"
// Ref: "files"."id" < "upload"."fid"

//// has_many :tag, through: :node_tag
//// Ref: "node"."tid" < "community_tags"."tid" < "community_tags"."id" 
//// Already has node.id to community_tags.nid above
Ref: "term_data"."id" < "community_tags"."tid"
Ref: "community_tags"."nid" - "community_tags"."tid"




//// BELONGS TO
//// Ref: "node"."uid" -> "user"."id"

// USERS
Ref: "users"."id" < "images"."uid"
Ref: "users"."id" < "node"."uid"
Ref: "users"."id" < "node_selections"."user_id"
Ref: "users"."id" < "node_revisions"."uid"
Ref: "users"."id" < "user_tags"."uid"
Ref: "users"."id" < "relationships"."follower_id"
Ref: "users"."id" < "relationships"."followed_id"
Ref: "users"."id" < "likes"."user_id"
Ref: "users"."id" < "answers"."uid"
Ref: "users"."id" < "answer_selections"."user_id"
Ref: "users"."id" < "comments"."uid"

//// THROUGH
//// has_many :revisions, through: :node
//// Ref: "users.vid" < "node"."vid" < "node_revisions"."id"
//// Already has node.id to node_revisions.nid above
Ref: "node_revisions"."id" < "node"."vid"
Ref: "node_revisions"."uid" - "node_revisions"."nid"


// ANSWERS
Ref: "answers"."id" < "answer_selections"."aid"
Ref: "answers"."id" < "comments"."aid"

//// BELONGS TO
//// Ref: "answers"."nid" -> "node"."id"
//// Ref: "answers"."uid" -> "user"."id"


// ANSWER SELECTIONS
//// BELONGS TO
//// Ref: "answer_selections"."aid" -> "answers"."id"
//// Ref: "answer_selections"."user_id" -> "user"."id"


//COMMENTS
Ref: "comments"."id" < "likes"."likeable_id"

//// BELONGS TO (ALREADY ABOVE)
//// Ref: "comments"."nid" -> "node"."id"
//// Ref: "comments"."uid" -> "user"."uid"
//// Ref: "comments"."aid" -> "answers"."id"

// FILES (ALREADY ABOVE)
//// BELONGS TO (REQUIRED FOR THROUGH)

// IMAGE
//// BELONGS TO (ALREADY ABOVE)
// belongs_to :user, foreign_key: :uid
// belongs_to :node, foreign_key: :nid

// LIKES (ALREADY ABOVE)

// NODE SELECTION (ALREADY ABOVE)

// TAGS (TERM_DATA)
Ref: "term_data"."id" < "tag_selections"."tid"
// Ref: "term_data"."id" < "community_tags"."tid" // ALREADY ABOVE

// COMMUNITY_TAGS (NODE_TAG, ALREADY ABOVE)
//// Ref: "community_tags"."id" -> "node"."tid"
//// Ref: "community_tags"."id" -> "term_data"."tid"
//// Ref: "community_tags"."id" -> "users"."uid"

// TAG_SELECTIONS (ALREADY ABOVE)
//Ref: "tag_selections"."tid" < "term_data"."id"

////BELONGS TO (ALREADY ABOVE)
////Ref: "tag_selections"."id" -> "term_data"."tid"

// USER TAGS
//// BELONGS TO
//// Ref: "user_tags"."uid" -> "user"."id"

// RELATIONSHIPS
//// SEE QUESTION IN NOTEBOOK
//// I BELIEVE IT'S COVERED ABOVE

// REVISION 
Ref: "node_revisions"."id" < "community_tags"."nid"
//Ref: "community_tags"."nid" < "community_tags"."tid" (ABOVE)
Ref: "community_tags"."tid" < "term_data"."vid"

// USER SELECTIONS
Ref: "user_selections"."self_id" - "users"."id"

//Ref: "likes"."id" < "likes"."likeable_type"

Ref: "tag_selections"."id" < "tag_selections"."tid"